In [1]:
#pickling

In [2]:
import numpy as np
import pandas as pd
import re
import requests
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import plotly as py
import cufflinks as cf
from plotly.offline import iplot

py.offline.init_notebook_mode(connected=True)
cf.go_offline()

In [4]:
df=pd.read_excel('review_labels.xlsx')
pd.set_option('max_colwidth',80)
df.head()

,Unnamed: 0,reviews,label
0,0,would say best available mic price range mic connected dslr smarthones lapto...,pos
1,1,hi using mic month writing review give genuine review mic let us begin begin...,pos
2,2,recording phone laptop quality get regular headphone watched review product ...,pos
3,3,stoped working within months recording starting record seconds records nothi...,neu
4,4,give honest review boya read full may help clear dilemma need depth review r...,pos


In [5]:
df.shape

(1000, 3)

In [6]:
# defining an empty data frame to amke a copy of data


In [7]:
# Converting Strings to Lowercase
df['reviews'] = [review.strip().lower() for review in df['reviews']]
df['reviews'][:5]

AttributeError: 'float' object has no attribute 'strip'

In [ ]:
contractions = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"

}

In [ ]:
def expand_contractions(text):
    for word in text.split():
        if word.lower() in contractions:
            text = text.replace(word, contractions[word.lower()])
    return text

In [ ]:
df['reviews'] = [expand_contractions(re.sub('’', "'", review)) for review in df['reviews']]
df['reviews'].head(5)

In [ ]:
import unicodedata
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    #https://docs.python.org/2/library/unicodedata.html
    return text

In [ ]:
df['reviews'] = [remove_accented_chars(review) for review in df['reviews']]
df['reviews'].head(5)

In [ ]:
# defining function to scrub special characters
def scrub_words(text):
    #Replace \xao characters in text
    text = re.sub('\xa0', ' ', text)
    
    #Replace non ascii / not words and digits
    text = re.sub("(\\W|\\d)",' ',text)
    
    #Replace new line characters and following text untill space
    text = re.sub('\n(\w*?)[\s]', '', text)
    
    #Remove html markup
    text = re.sub("<.*?>", ' ', text)
    
    #Remove extra spaces from the text
    text = re.sub("\s+", ' ', text)
    return text

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
df['reviews'] = [scrub_words(review) for review in df['reviews']]
df['reviews'].head(3)

### <font> <font color="red"> Checking for NULL_VALUES

In [ ]:
df.isna().sum()

### <font> <font color="red"> Removing stop words from df and applying lemmatization

In [ ]:
# Importing spacy
import spacy


# Loading model
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])

# Lemmatization with stopwords removal
df['review_lemm']=df['reviews'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))

In [ ]:
df['review_lemm'].head(2)

 ##  <font> <font color="red"> using VADER Sentiment Analysis on reviews to assign labels

In [ ]:
# Import SentimentIntensityAnalyzer and create an sid object
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [ ]:
df2=df.drop(['rating','rate_pt'],axis=1)
df2.head(2)

In [ ]:
df2['score'] = df2['reviews'].apply(lambda reviews:sid.polarity_scores(reviews))
#df2.head(3)

In [ ]:
df2['compound']  = df2['score'].apply(lambda score_dict: score_dict['compound'])
#df2.head(3)

In [ ]:
df2['label'] = df2['compound'].apply(lambda c: 'pos' if c >0.05 else('neu'if  c<=0.050 and c>=-0.05 else 'neg'))

In [ ]:
df2.head()

In [ ]:
df2.drop(['score','compound','review_lemm'],inplace=True,axis=1)
#df2.head()

In [ ]:
df3=pd.DataFrame()
df3=df2.copy()
df3.to_excel("Amazon_labels_3.xlsx",index=False)